In [32]:
pip install pandas openpyxl shapely


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
from shapely.geometry import Point, Polygon
from shapely.wkt import loads as load_wkt

# 1. Coordenadas del polígono del Distrito 5
coordenadas_distrito_19 = {
   "districto19": load_wkt ("POLYGON ((-65.2221135 -26.8577649, -65.2232936 -26.8572289, -65.2337221 -26.8656132, -65.2302459 -26.8682165, -65.2303747 -26.8744564, -65.2178005 -26.8765235, -65.2068785 -26.8636033, -65.2194527 -26.8609808, -65.219045 -26.8589134, -65.2221135 -26.8577649))")
}

coordenadas=coordenadas_distrito_19["districto19"]
poligono = Polygon(coordenadas)

# 2. Leer el archivo
df = pd.read_excel("datos_originales.xlsx")

# 3. Limpiar nombres de columnas
df.columns = df.columns.str.strip()

# 4. Forzar conversión de lat/lon a float (coercing errores como "Error" a NaN)
df["Latitud"] = pd.to_numeric(df["Latitud"], errors="coerce")
df["Longitud"] = pd.to_numeric(df["Longitud"], errors="coerce")

# 5. Eliminar filas con lat/lon inválidos
df = df.dropna(subset=["Latitud", "Longitud"])

# 6. Verificar si cada punto está dentro del polígono
df["Dentro Distrito 19"] = df.apply(
    lambda row: poligono.contains(Point(row["Longitud"], row["Latitud"])),
    axis=1
)

# 7. Filtrar puntos que están dentro
df_distrito4 = df[df["Dentro Distrito 19"]].drop(columns=["Dentro Distrito 19"])

# 8. Exportar resultados
df_distrito4.to_excel("puntos_dentro_distrito19.xlsx", index=False)

print(f"✅ Se exportaron {len(df_distrito4)} puntos dentro del Distrito 19.")



✅ Se exportaron 33 puntos dentro del Distrito 19.


In [3]:
import pandas as pd

df = pd.read_excel("datos_originales.xlsx")
print(df.columns.tolist())


['Fecha', 'Direccion', 'Procedimiento Destacado', 'Latitud ', 'Longitud']


In [3]:
pip install geopy


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import pandas as pd
from geopy.distance import geodesic

# 1. Cargar el archivo filtrado previamente
archivo = "puntos_dentro_distrito19.xlsx"
df = pd.read_excel(archivo)

# 2. Asegurar que no haya espacios en los nombres de columna
df.columns = df.columns.str.strip()

# 3. Separar los relevamientos y los otros
df_relevamiento = df[df["Procedimiento Destacado"].str.contains("Relevamiento", case=False, na=False)].copy()
df_otros = df[~df.index.isin(df_relevamiento.index)].copy()

# 4. Inicializar columna para marcar agrupados
df_relevamiento["agrupado"] = False
agrupados = []

# 5. Definir la tolerancia (en metros) para puntos cercanos
tolerancia_metros = 15

# 6. Agrupar puntos cercanos entre sí
for idx, row in df_relevamiento.iterrows():
    if df_relevamiento.at[idx, "agrupado"]:
        continue

    punto_base = (row["Latitud"], row["Longitud"])
    grupo = [idx]
    df_relevamiento.at[idx, "agrupado"] = True

    for jdx, row2 in df_relevamiento.iterrows():
        if idx == jdx or df_relevamiento.at[jdx, "agrupado"]:
            continue

        punto_comp = (row2["Latitud"], row2["Longitud"])
        distancia = geodesic(punto_base, punto_comp).meters

        if distancia <= tolerancia_metros:
            grupo.append(jdx)
            df_relevamiento.at[jdx, "agrupado"] = True

    fila_resultado = df_relevamiento.loc[idx].copy()
    fila_resultado["Cantidad afectada"] = len(grupo)
    agrupados.append(fila_resultado)

# 7. Crear DataFrame con relevamientos agrupados
df_relevamientos_agrupados = pd.DataFrame(agrupados).drop(columns=["agrupado"])

# 8. Para los otros procedimientos, asignar "Cantidad afectada" = 1
df_otros["Cantidad afectada"] = 1

# 9. Unir relevamientos agrupados con los demás procedimientos
df_final = pd.concat([df_relevamientos_agrupados, df_otros], ignore_index=True)

# 10. Exportar archivo final
df_final.to_excel("puntos_con_cantidad_afectada_districto19.xlsx", index=False)

print(f"✅ Archivo generado: puntos_con_cantidad_afectada_districto5.xlsx (total: {len(df_final)} filas)")


✅ Archivo generado: puntos_con_cantidad_afectada_districto5.xlsx (total: 31 filas)


In [43]:
pip install pandas shapely openpyxl


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
from shapely.geometry import Point
from shapely.wkt import loads as load_wkt

poligonos = {
    "San Alberto": load_wkt("POLYGON ((-65.223368 -26.873962, -65.22215 -26.871274, -65.221655 -26.869594, -65.229601 -26.868538, -65.229562 -26.869313, -65.229597 -26.87067, -65.229219 -26.870695, -65.229152 -26.870704, -65.228251 -26.870834, -65.227865 -26.870889, -65.227606 -26.869576, -65.223624 -26.870139, -65.224598 -26.871316, -65.22494 -26.871916, -65.226046 -26.873478, -65.224774 -26.873708, -65.223368 -26.873962))"),

    "Parodi": load_wkt("POLYGON ((-65.221138 -26.866482, -65.221655 -26.869594, -65.219916 -26.86992, -65.212761 -26.871263, -65.210833 -26.868323, -65.213417 -26.867888, -65.217925 -26.867067, -65.221138 -26.866482))"),

    "Villa Angelina": load_wkt("POLYGON ((-65.212761 -26.871263, -65.220454 -26.869819, -65.221655 -26.869594, -65.22215 -26.871274, -65.222536 -26.872128, -65.22091 -26.872434, -65.220911 -26.872431, -65.214509 -26.873649, -65.214284 -26.873561, -65.212761 -26.871263))"),

    "Jose Hernandez": load_wkt("POLYGON ((-65.227802 -26.870574, -65.227941 -26.871288, -65.227964 -26.871407, -65.228064 -26.871924, -65.228285 -26.873074, -65.227021 -26.873302, -65.226391 -26.873416, -65.226046 -26.873478, -65.225724 -26.873016, -65.225411 -26.872576, -65.225217 -26.872304, -65.22494 -26.871916, -65.224598 -26.871316, -65.223624 -26.870139, -65.224859 -26.869961, -65.226273 -26.869752, -65.227606 -26.869576, -65.227802 -26.870574))"),

    "CRUCERO ARA GENERAL BELGRANO": load_wkt("POLYGON ((-65.223149 -26.873483, -65.223368 -26.873962, -65.224216 -26.875831, -65.223453 -26.875972, -65.216726 -26.877243, -65.215192 -26.874861, -65.221256 -26.873711, -65.223083 -26.873338, -65.223149 -26.873483))"),
    "La Canchita": load_wkt("POLYGON ((-65.223006 -26.873168, -65.223083 -26.873338, -65.221275 -26.873707, -65.221215 -26.8735, -65.221122 -26.873173, -65.221035 -26.872869, -65.22091 -26.872434, -65.222536 -26.872128, -65.222814 -26.872743, -65.223006 -26.873168))"),
    "Palmera Norte":load_wkt("POLYGON ((-65.229618 -26.870649, -65.229584 -26.871203, -65.229571 -26.871564, -65.229573 -26.872601, -65.229579 -26.873521, -65.229581 -26.874367, -65.229757 -26.874804, -65.227514 -26.875233, -65.226138 -26.875483, -65.225327 -26.87563, -65.224433 -26.875792, -65.224216 -26.875831, -65.223957 -26.875245, -65.223679 -26.874618, -65.223368 -26.873962, -65.224774 -26.873708, -65.226046 -26.873478, -65.226391 -26.873416, -65.227021 -26.873302, -65.228285 -26.873074, -65.228064 -26.871924, -65.227964 -26.871407, -65.227865 -26.870889, -65.228251 -26.870834, -65.229152 -26.870704, -65.229219 -26.870695, -65.229618 -26.870649))"),
    "Ampliacion Villa Angelina":load_wkt("POLYGON ((-65.220905 -26.872441, -65.221268 -26.873701, -65.215192 -26.874861, -65.214504 -26.873715, -65.214509 -26.873649, -65.220905 -26.872441))"),
    "Villa Marisa":load_wkt("POLYGON ((-65.220905 -26.872441, -65.221268 -26.873701, -65.215192 -26.874861, -65.214504 -26.873715, -65.214509 -26.873649, -65.220905 -26.872441))")
}


   

# --- 2. Cargar archivo con puntos ---
df = pd.read_excel("puntos_con_cantidad_afectada_districto19.xlsx")
df.columns = df.columns.str.strip()
df["Latitud"] = pd.to_numeric(df["Latitud"], errors="coerce")
df["Longitud"] = pd.to_numeric(df["Longitud"], errors="coerce")
df = df.dropna(subset=["Latitud", "Longitud"])

# --- 3. Asignar barrio a cada punto ---
def asignar_barrio(lat, lon):
    punto = Point(lon, lat)
    for nombre_barrio, poligono in poligonos.items():
        if poligono.contains(punto):
            return nombre_barrio
    return "Fuera de los barrios"

df["Barrio"] = df.apply(lambda row: asignar_barrio(row["Latitud"], row["Longitud"]), axis=1)

# --- 4. Exportar resultado ---
df.to_excel("puntos_con_barrios_districto19.xlsx", index=False)
print(f"✅ Archivo exportado con {len(df)} puntos y columna 'Barrio' agregada.")


✅ Archivo exportado con 31 puntos y columna 'Barrio' agregada.


In [6]:
import pandas as pd
import re

# 1. Cargar archivo de puntos ya procesado con barrios
df = pd.read_excel("puntos_con_barrios_districto19.xlsx")

# 2. Función para extraer el nombre de la calle principal
def extraer_calle(direccion):
    direccion = str(direccion).strip()
    # Caso intersección → lo que está antes del " y "
    if " y " in direccion.lower():
        return direccion.split(" y ")[0].strip().title()
    # Caso dirección con número
    match = re.match(r"^(.*?)(\s\d+\b)", direccion)
    if match:
        return match.group(1).strip().title()
    # Si no hay número ni intersección → devolver todo hasta la coma
    return direccion.split(",")[0].strip().title()

# 3. Función para extraer la segunda calle o número de la dirección
def extraer_interseccion(direccion):
    direccion = str(direccion).strip()
    # Caso intersección → lo que está después del " y "
    if " y " in direccion.lower():
        return direccion.split(" y ")[1].split(",")[0].strip().title()
    # Caso dirección con número
    partes = direccion.split(",")[0].split()
    if partes:
        ultima = partes[-1]
        return ultima if ultima.isdigit() else ""
    return ""

# 4. Aplicar ambas funciones
df["Direccion_Calle"] = df["Direccion"].apply(extraer_calle)
df["Direccion_Numero_Interseccion"] = df["Direccion"].apply(extraer_interseccion)

# 5. Crear DataFrame final con las columnas formateadas
df_final = pd.DataFrame({
    "ID_Punto": "",
    "Tipo_Intervencion_ID": "18",
    "Subtipo_Intervencion_ID": "todos",
    "Descripcion_Detallada": df["Procedimiento Destacado"],
    "Latitud": df["Latitud"],
    "Longitud": df["Longitud"],
    "Direccion_Calle": df["Direccion_Calle"],
    "Direccion_Numero_Interseccion": df["Direccion_Numero_Interseccion"],
    "Barrio_ID": df["Barrio"],
    "Distrito_ID": "19",
    "Fecha_Realizacion": df["Fecha"],
    "Cantidad_Afectada": df["Cantidad afectada"],
    "Unidad_Medida": "locales gastronómicos",
    "Estado_Actual": "completados",
    "Responsable_Direccion_ID": "18",
    "Notas_Adicionales": ""
})

# 6. Exportar a Excel
df_final.to_excel("intervenciones_formato_final_districto19.xlsx", index=False)
print("✅ Archivo final exportado: 'intervenciones_formato_final_districto19.xlsx'")




✅ Archivo final exportado: 'intervenciones_formato_final_districto19.xlsx'


In [20]:
pip install folium pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import folium

# 1. Cargar el archivo con datos ya procesados
df = pd.read_excel("intervenciones_formato_final_districto4.xlsx")

# 2. Crear el mapa centrado en San Miguel de Tucumán
mapa = folium.Map(location=[-26.833, -65.216], zoom_start=13)

# 3. Agregar cada punto como marcador circular
for _, row in df.iterrows():
    direccion = row.get("Direccion_Calle", "Sin dirección")
    barrio = row.get("Barrio_ID", "Sin barrio")

    folium.CircleMarker(
        location=[row["Latitud"], row["Longitud"]],
        radius=5,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.6,
        popup=folium.Popup(f"{direccion}<br><b>Barrio:</b> {barrio}", max_width=300)
    ).add_to(mapa)

# 4. Guardar el mapa en un archivo HTML
mapa.save("mapa_inspecciones_districto4.html")
print("✅ Mapa exportado como 'mapa_inspecciones_districto4.html'")


✅ Mapa exportado como 'mapa_inspecciones_districto14.html'


In [22]:
pip install shapely folium pandas


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import pandas as pd
import folium
from shapely.wkt import loads as load_wkt
from shapely.geometry import mapping

# 1. Leer puntos desde archivo
df = pd.read_excel("intervenciones_formato_final_districto4.xlsx")

# 2. Crear mapa centrado en San Miguel de Tucumán
mapa = folium.Map(location=[-26.833, -65.216], zoom_start=13)

# 3. Agregar puntos
for _, row in df.iterrows():
    direccion = row.get("Direccion_Calle", "Sin dirección")
    barrio = row.get("Barrio_ID", "Sin barrio")

    folium.CircleMarker(
        location=[row["Latitud"], row["Longitud"]],
        radius=5,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.6,
        popup=folium.Popup(f"{direccion}<br><b>Barrio:</b> {barrio}", max_width=300)
    ).add_to(mapa)

# 4. Agregar polígonos de barrios
for nombre, poligono in poligonos.items():
    coords = [[lat, lon] for lon, lat in poligono.exterior.coords]
    folium.Polygon(
        locations=coords,
        color="red",
        fill=True,
        fill_color="red",
        fill_opacity=0.1,
        weight=2,
        popup=f"{nombre}"
    ).add_to(mapa)

# 5. Guardar como archivo HTML
mapa.save("mapa_con_poligonos.html")
print("✅ Mapa generado como 'mapa_con_poligonos4.html'")


✅ Mapa generado como 'mapa_con_poligonos14.html'


In [ ]:
import pandas as pd
import folium
from shapely.wkt import loads as load_wkt
from shapely.geometry import MultiPolygon
from folium.features import DivIcon

# ... tu código de puntos arriba ...
# 1. Leer puntos desde archivo
df = pd.read_excel("intervenciones_formato_final_districto14.xlsx")

# 2. Crear mapa centrado en San Miguel de Tucumán
mapa = folium.Map(location=[-26.833, -65.216], zoom_start=13)
# Diccionario usando load_wkt (ya importado)
poligonos = {
    "VICTORIA": load_wkt("POLYGON ((-65.226925 -26.845078, -65.227346 -26.845, -65.235775 -26.843421, -65.236759 -26.843203, -65.237089 -26.843589, -65.237507 -26.844384, -65.23775 -26.844844, -65.237813 -26.845073, -65.236255 -26.845395, -65.235709 -26.845515, -65.235828 -26.846164, -65.237994 -26.845724, -65.238298 -26.846816, -65.238552 -26.848017, -65.236887 -26.848364, -65.236662 -26.847523, -65.233575 -26.848159, -65.233903 -26.849511, -65.235311 -26.849183, -65.235472 -26.849664, -65.237202 -26.849301, -65.237362 -26.849951, -65.237762 -26.851571, -65.23606 -26.85185, -65.23575 -26.850577, -65.234295 -26.850851, -65.232543 -26.851175, -65.228823 -26.85187, -65.224369 -26.852702, -65.223722 -26.850004, -65.222748 -26.845945, -65.226925 -26.845078))"),
    "EL CRUCE": load_wkt("POLYGON ((-65.219902 -26.846491, -65.218612 -26.841067, -65.221478 -26.840454, -65.222748 -26.845945, -65.219902 -26.846491))"),
    "AVENIDA": load_wkt("POLYGON ((-65.223722 -26.850004, -65.221925 -26.850366, -65.221634 -26.848989, -65.221076 -26.84911, -65.220502 -26.846376, -65.222748 -26.845945, -65.223722 -26.850004))"),
    "COPIAT II": load_wkt("POLYGON ((-65.232904 -26.852454, -65.232543 -26.851175, -65.23575 -26.850577, -65.23606 -26.85185, -65.232904 -26.852454))"),
    "SANTA RITA": load_wkt("POLYGON ((-65.238048 -26.852816, -65.232163 -26.853948, -65.231877 -26.85265, -65.23606 -26.85185, -65.237762 -26.851571, -65.238048 -26.852816))"),
    "LOS VAZQUEZ": load_wkt("POLYGON ((-65.224163 -26.855491, -65.223682 -26.855582, -65.223521 -26.854914, -65.223356 -26.854232, -65.223237 -26.853738, -65.221815 -26.853999, -65.220941 -26.850543, -65.221925 -26.850366, -65.223722 -26.850004, -65.224369 -26.852702, -65.22652 -26.8523, -65.226777 -26.853519, -65.227076 -26.85494, -65.226351 -26.85508, -65.224163 -26.855491))"),
    "SUTIAGA": load_wkt("POLYGON ((-65.240412 -26.856651, -65.235798 -26.857524, -65.23552 -26.85606, -65.240087 -26.85523, -65.240412 -26.856651))"),
    "1° DE MAYO (VICENTE HARO)": load_wkt("POLYGON ((-65.239048 -26.849599, -65.240041 -26.851712, -65.240242 -26.852138, -65.240117 -26.852378, -65.239583 -26.852524, -65.238048 -26.852816, -65.237762 -26.851571, -65.237682 -26.851248, -65.237547 -26.8507, -65.237362 -26.849951, -65.239048 -26.849599))"),
    "AMPLIACION VICTORIA": load_wkt("POLYGON ((-65.236662 -26.847523, -65.236887 -26.848364, -65.238552 -26.848017, -65.238754 -26.848973, -65.239048 -26.849599, -65.237362 -26.849951, -65.237202 -26.849301, -65.235472 -26.849664, -65.235311 -26.849183, -65.233922 -26.849508, -65.233786 -26.848919, -65.233616 -26.848154, -65.236662 -26.847523))"),
    "SANTA ELENA": load_wkt("POLYGON ((-65.240412 -26.856651, -65.240564 -26.857399, -65.238173 -26.857818, -65.237778 -26.85788, -65.237406 -26.857954, -65.236017 -26.858211, -65.23523 -26.858355, -65.234702 -26.858427, -65.234619 -26.857762, -65.234483 -26.857026, -65.234341 -26.856274, -65.23552 -26.85606, -65.235798 -26.857524, -65.23727 -26.857268, -65.238978 -26.856922, -65.240412 -26.856651))"),
    "AVAREZ CONDARCO (CHAÑARITOS)": load_wkt("POLYGON ((-65.227051 -26.858302, -65.227864 -26.858871, -65.228592 -26.859403, -65.228817 -26.859562, -65.228855 -26.859588, -65.229431 -26.859994, -65.230543 -26.860734, -65.23143 -26.861495, -65.231555 -26.861673, -65.231682 -26.861857, -65.231541 -26.861882, -65.228643 -26.862342, -65.228279 -26.861736, -65.228048 -26.861445, -65.227732 -26.861079, -65.227214 -26.860543, -65.226589 -26.860012, -65.225973 -26.85949, -65.225096 -26.858747, -65.227051 -26.858302))"),
    "SAN ANIBAL": load_wkt("POLYGON ((-65.233065 -26.857819, -65.234542 -26.85752, -65.234596 -26.857824, -65.234674 -26.858267, -65.234702 -26.858427, -65.234859 -26.859113, -65.235044 -26.859817, -65.235162 -26.860317, -65.235327 -26.861021, -65.235464 -26.861689, -65.235899 -26.863664, -65.23428 -26.865022, -65.233945 -26.865303, -65.233086 -26.863978, -65.231657 -26.861818, -65.23143 -26.861495, -65.230543 -26.860734, -65.229955 -26.86036, -65.228817 -26.859562, -65.227864 -26.858871, -65.233065 -26.857819))"),
    "EL GRAFICO I": load_wkt("POLYGON ((-65.234341 -26.856274, -65.234462 -26.857021, -65.234542 -26.85752, -65.233065 -26.857819, -65.229235 -26.858594, -65.227864 -26.858871, -65.227051 -26.858302, -65.226293 -26.857771, -65.226972 -26.857648, -65.228285 -26.857412, -65.229654 -26.857146, -65.2306 -26.85697, -65.232767 -26.856567, -65.234341 -26.856274))"),
    "PRESIDENTE PERON (VERA)": load_wkt("POLYGON ((-65.221909 -26.854367, -65.221815 -26.853999, -65.223237 -26.853738, -65.223356 -26.854232, -65.223521 -26.854914, -65.223663 -26.85548, -65.223682 -26.855582, -65.223111 -26.855635, -65.222392 -26.855702, -65.222048 -26.854751, -65.22194 -26.854454, -65.221909 -26.854367))"),
    "JUAN B. TERAN": load_wkt("POLYGON ((-65.231318 -26.851427, -65.232543 -26.851175, -65.232904 -26.852454, -65.231877 -26.85265, -65.232163 -26.853948, -65.229363 -26.854494, -65.227076 -26.85494, -65.226994 -26.854548, -65.226777 -26.853519, -65.226633 -26.852834, -65.22652 -26.8523, -65.227193 -26.852174, -65.228466 -26.851937, -65.229811 -26.851685, -65.231318 -26.851427))"),
    "11 DE MARZO": load_wkt("POLYGON ((-65.232163 -26.853948, -65.234904 -26.85342, -65.236345 -26.853143, -65.238048 -26.852816, -65.239583 -26.852524, -65.239729 -26.853308, -65.23988 -26.853958, -65.23998 -26.854654, -65.240087 -26.85523, -65.236981 -26.855794, -65.23552 -26.85606, -65.234341 -26.856274, -65.232767 -26.856567, -65.231987 -26.856712, -65.2306 -26.85697, -65.229654 -26.857146, -65.228285 -26.857412, -65.226972 -26.857648, -65.226293 -26.857771, -65.227051 -26.858302, -65.225486 -26.858658, -65.225096 -26.858747, -65.224416 -26.858177, -65.223736 -26.857607, -65.223112 -26.856999, -65.222816 -26.856466, -65.222708 -26.856271, -65.222392 -26.855702, -65.223111 -26.855635, -65.223682 -26.855582, -65.225007 -26.855332, -65.226351 -26.85508, -65.227756 -26.854808, -65.229136 -26.854538, -65.230404 -26.854291, -65.232163 -26.853948))"),
    "SAN FERNANDO": load_wkt("POLYGON ((-65.241236 -26.856495, -65.24173 -26.858547, -65.24182 -26.858921, -65.2417 -26.859015, -65.241326 -26.859294, -65.240757 -26.859754, -65.239906 -26.860457, -65.238999 -26.861141, -65.235899 -26.863664, -65.234859 -26.859113, -65.234702 -26.858427, -65.240564 -26.857399, -65.239583 -26.852524, -65.240117 -26.852378, -65.240242 -26.852138, -65.241236 -26.856495))"),
    "NORGAS": load_wkt("POLYGON ((-65.2206 -26.849166, -65.22081 -26.849141, -65.221076 -26.84911, -65.221634 -26.848989, -65.221925 -26.850366, -65.220941 -26.850543, -65.220726 -26.849676, -65.2206 -26.849166))"),
    "JORGE LUIS BORGES": load_wkt("POLYGON ((-65.237813 -26.845073, -65.237994 -26.845724, -65.236495 -26.846029, -65.235828 -26.846164, -65.235794 -26.845975, -65.235755 -26.845764, -65.235709 -26.845515, -65.236299 -26.845386, -65.23657 -26.84533, -65.236761 -26.84529, -65.236924 -26.845257, -65.237113 -26.845218, -65.237288 -26.845181, -65.237813 -26.845073))"),
    "BERNABE ARAOZ": load_wkt("POLYGON ((-65.220502 -26.846376, -65.220992 -26.848712, -65.221076 -26.84911, -65.22081 -26.849141, -65.2206 -26.849166, -65.220302 -26.847804, -65.219902 -26.846491, -65.220095 -26.846454, -65.220502 -26.846376))"),
    "23 DE FEBRERO": load_wkt("POLYGON ((-65.233067 -26.863983, -65.231248 -26.86438, -65.229963 -26.864655, -65.228643 -26.862342, -65.231682 -26.861861, -65.233067 -26.863983))"),
    "CIUDADELA SUR": load_wkt("POLYGON ((-65.221478 -26.840454, -65.228515 -26.839078, -65.229154 -26.8413, -65.225968 -26.842307, -65.226925 -26.845078, -65.222748 -26.845945, -65.221478 -26.840454))")
}
# 3. Agregar puntos
for _, row in df.iterrows():
    direccion = row.get("Direccion_Calle", "Sin dirección")
    barrio = row.get("Barrio_ID", "Sin barrio")

    folium.CircleMarker(
        location=[row["Latitud"], row["Longitud"]],
        radius=5,
        color="blue",
        fill=True,
        fill_color="blue",
        fill_opacity=0.6,
        popup=folium.Popup(f"{direccion}<br><b>Barrio:</b> {barrio}", max_width=300)
    ).add_to(mapa)

# Dibujar polígonos + etiquetas (soporta MultiPolygon)
for nombre, poligono in poligonos.items():
    partes = list(poligono.geoms) if isinstance(poligono, MultiPolygon) else [poligono]

    for g in partes:
        coords = [[lat, lon] for lon, lat in g.exterior.coords]  # folium requiere [lat, lon]
        folium.Polygon(
            locations=coords,
            color="red",
            fill=True,
            fill_color="red",
            fill_opacity=0.1,
            weight=2,
            popup=nombre
        ).add_to(mapa)

    # etiqueta (representative_point evita quedar fuera en formas raras)
    p = poligono.representative_point()
    folium.Marker(
        location=[p.y, p.x],
        icon=DivIcon(
            icon_size=(100, 36),
            icon_anchor=(0, 0),
            html=f"""
            <div style="
                background: rgba(255,0,0);
                padding: 2px 6px;
                border-radius: 6px;
                border: 1px solid rgba(0,0,0,0.2);
                font-size: 9px;
                color:  rgba(0, 0, 0);
                font-weight: 600; 
                font-style= bold;
                white-space: nowrap;">
                {nombre}
            </div>
            """
        )
    ).add_to(mapa)

mapa.save("mapa_con_poligonos_final.html")


In [13]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Identifícate claramente (proyecto + email o URL de contacto)
geolocator = Nominatim(
    user_agent="bromatologia-smt/1.0 (contacto: tu_email@dominio.gob.ar)",
    timeout=10
)

# Respetar TOS: 1 seg entre consultas
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

direccion = "Av. América 1734, San Miguel de Tucumán, Tucumán"
ubicacion = geocode(direccion)

if ubicacion:
    print(ubicacion.address)
    print(ubicacion.latitude, ubicacion.longitude)
else:
    print("No se encontró la dirección")



Avenida América, Feput, San Miguel de Tucumán, Departamento Capital, Tucumán, T4000, Argentina
-26.7872288 -65.243043
